1) Write a program to generate Hash code for a given text using SHA 512

In [ ]:
import hashlib

text = input("Enter text: ")
hash_code = hashlib.sha512(text.encode()).hexdigest()

print("SHA-512 Hash:", hash_code)

Enter text: helloworld
SHA-512 Hash: 1594244d52f2d8c12b142bb61f47bc2eaf503d6d9ca8480cae9fcf112f66e4967dc5e8fa98285e36db8af1b8ffa8b84cb15e0fbcf836c3deb803c13f37659a60


2) Write a program to send a message ensuring confidentiality and integrity
Hint for 2:

i) perform diffie hellman exchange (from previous lab) to generate a session key

ii) Compute Hash code of the message text

iii) use AES to encrypt the (text + hash code)

iv) encrypt the (message + Hash Code)

v) send it to client/server

vi) decrypt the message

vii) check the integrity.

In [ ]:
import hashlib
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend

prv_A = ec.generate_private_key(ec.SECP256R1())
prv_B = ec.generate_private_key(ec.SECP256R1())

pub_A = prv_A.public_key()
pub_B = prv_B.public_key()

sec_A = prv_A.exchange(ec.ECDH(), pub_B)
sec_B = prv_B.exchange(ec.ECDH(), pub_A)

assert sec_A == sec_B, "Key exchange failed!"
aes_k = sec_A[:16]

def get_hash(msg):
    return hashlib.sha512(msg.encode()).hexdigest()

def enc_msg(msg, key):
    msg_hash = get_hash(msg)
    data = (msg + msg_hash).encode()

    pad = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = pad.update(data) + pad.finalize()

    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    enc = cipher.encryptor()

    return enc.update(padded_data) + enc.finalize()

def dec_msg(ct, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    dec = cipher.decryptor()

    dec_data = dec.update(ct) + dec.finalize()

    unpad = padding.PKCS7(algorithms.AES.block_size).unpadder()
    data = unpad.update(dec_data) + unpad.finalize()

    msg = data[:-128].decode().strip()
    rcv_hash = data[-128:].decode().strip()

    return msg, "Integrity Verified!" if get_hash(msg) == rcv_hash else "Integrity Check Failed!"

msg = "Hello, Secure World!"
ct = enc_msg(msg, aes_k)

print("\nCiphertext:", ct.hex())

dec_msg, status = dec_msg(ct, aes_k)

print("\nDecrypted Message:", dec_msg)
print("Integrity Status:", status)



Ciphertext: a7f0335b8d3edc3df2f996ece52ea56ac442c3827c8ff40ded4191129529a6ae009d8852c7eb3546b0a4a65d69a3332792b10f7e41b082d8eea0ced0adb4ac8f49378121738c78e5ce4dd35150385c51057b26d6a10c1d90bd20e7a69f1640db7eba9a898c7995b9f4582a4de70003cd9980fc9d0516076227d595d466139fafff010fa61fd7a63b49468606978b529f20f673cfb8559b255275274920db0031

Decrypted Message: Hello, Secure World!
Integrity Status: Integrity Verified!
